In [ ]:
#A dataset is imported from Kaggle containing diseases and their symptoms
#Preprocessing the dataset
#import the dataset
import pandas as pd
data = pd.read_csv('/content/Symptom2Disease.csv')
#remove punctuation, lowercase the text, and handle special characters.
import re
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    return text.lower()
#view the original column names
print("Original column names:")
print(data.columns)

#rename columns
column_mapping = {
    'text': 'symptoms',
    'label': 'disease'
}

data.rename(columns=column_mapping, inplace=True)

# View the updated column names
print("\nUpdated column names:")
print(data.columns)

# Save the updated dataset (optional)
data.to_csv('updated_dataset.csv', index=False)
data['symptoms_cleaned'] = data['symptoms'].apply(clean_text)


Original column names:
Index(['Unnamed: 0', 'label', 'text'], dtype='object')

Updated column names:
Index(['Unnamed: 0', 'disease', 'symptoms'], dtype='object')


In [ ]:
#using TF-IDF or Word Embeddings to represent the symptoms text numerically
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000)  # Adjust max_features as needed
X = vectorizer.fit_transform(data['symptoms_cleaned'])

In [ ]:
#converting diseases to numeric labels
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['disease'])

In [ ]:
#spliting the dataset into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#using logistic regression to train the model
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
#evaluating the model using accuracy, precision, recall, F1-score to evaluate the model on the test set
from sklearn.metrics import classification_report
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

                                 precision    recall  f1-score   support

                           Acne       1.00      1.00      1.00         7
                      Arthritis       1.00      1.00      1.00        10
               Bronchial Asthma       1.00      1.00      1.00        11
           Cervical spondylosis       1.00      1.00      1.00         7
                    Chicken pox       1.00      0.92      0.96        12
                    Common Cold       1.00      1.00      1.00        12
                         Dengue       1.00      0.83      0.91        12
          Dimorphic Hemorrhoids       1.00      1.00      1.00         7
               Fungal infection       1.00      1.00      1.00        13
                   Hypertension       1.00      1.00      1.00        10
                       Impetigo       1.00      1.00      1.00        11
                       Jaundice       1.00      1.00      1.00        11
                        Malaria       1.00      1.

In [ ]:
#saving the trained model and vectorizer for future use
import pickle
pickle.dump(model, open('disease_model.pkl', 'wb'))
pickle.dump(vectorizer, open('vectorizer.pkl', 'wb'))

In [ ]:
#
from flask import Flask, request, jsonify
import pickle
pickle.dump(label_encoder, open('label_encoder.pkl', 'wb'))
app = Flask(__name__)

# Load the model and vectorizer
model = pickle.load(open('disease_model.pkl', 'rb'))
vectorizer = pickle.load(open('vectorizer.pkl', 'rb'))
label_encoder = pickle.load(open('label_encoder.pkl', 'rb'))

@app.route('/predict', methods=['POST'])
def predict_disease():
    symptoms = request.json.get('symptoms')
    symptoms_cleaned = clean_text(symptoms)
    symptoms_vectorized = vectorizer.transform([symptoms_cleaned])
    prediction = model.predict(symptoms_vectorized)
    disease = label_encoder.inverse_transform(prediction)
    return jsonify({'disease': disease[0]})

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
import requests
def get_nearby_centers(location, radius=5000, type='hospital'):
    api_key = "YOUR_API_KEY"
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location}&radius={radius}&type={type}&key={api_key}"
    response = requests.get(url)
    return response.json()

In [ ]:
from flask import Flask, request, jsonify
app = Flask(__name__)

@app.route('/get_advice', methods=['POST'])
def get_advice():
    symptoms = request.json.get('symptoms')
    # Process symptoms and return advice
    return jsonify({"advice": "Consult a doctor if symptoms persist."})

@app.route('/get_centers', methods=['GET'])
def get_centers():
    location = request.args.get('location')
    # Fetch and return healthcare centers
    return jsonify(get_nearby_centers(location))

if __name__ == "__main__":
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
#frontend
fetch('/get_advice', {
    method: 'POST',
    headers: {'Content-Type': 'application/json'},
    body: JSON.stringify({symptoms: 'fever, cough'})
})
.then(response => response.json())
.then(data => console.log(data.advice));

SyntaxError: invalid syntax (<ipython-input-18-2dc22ab85d32>, line 6)

In [ ]:
import requests

def get_prediction():
    # Flask API URL
    url = "http://127.0.0.1:5000/predict"

    print("Welcome to the Healthcare Chatbot!")
    print("Enter your symptoms below (type 'exit' to quit):")

    while True:
        # Get symptoms from user
        symptoms = input("Enter symptoms: ")

        if symptoms.lower() == "exit":
            print("Exiting the chatbot. Stay healthy!")
            break

        try:
            # Send a POST request to the Flask API
            response = requests.post(
                url,
                json={"symptoms": symptoms},
                headers={"Content-Type": "application/json"}
            )

            # Check the response
            if response.status_code == 200:
                prediction = response.json().get("disease", "No prediction available")
                print(f"Predicted Disease: {prediction}\n")
            else:
                print(f"Error {response.status_code}: {response.text}")

        except requests.exceptions.ConnectionError:
            print("Error: Could not connect to the Flask server. Is it running?")
            break
        except Exception as e:
            print(f"An error occurred: {e}")
            break

if __name__ == "__main__":
    get_prediction()

Welcome to the Healthcare Chatbot!
Enter your symptoms below (type 'exit' to quit):
Enter symptoms: fever
Error: Could not connect to the Flask server. Is it running?
